In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.io import loadmat
import pywt

# ====================== 配置参数 ======================
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 数据参数
SNR_dB = 20
ADD_NOISE = True
ADD_DOPPLER = True
FS = 20e6
FC = 2.4e9
VELOCITY_KMH = 120

# Wavelet 特征
USE_LOG = True
WAVELET = 'db6'
WAVELET_LEVEL = 6

# 训练参数
BATCH_SIZE = 64
EPOCHS = 200
LR = 3e-4
WEIGHT_DECAY = 1e-5
N_SPLITS = 5
PATIENCE = 10
DROPOUT_RATE = 0

# 保存路径
SAVE_ROOT = "./training_results"
os.makedirs(SAVE_ROOT, exist_ok=True)

# ====================== 数据读取 ======================
DATA_FOLDER = r"..\los_data"  # Wi-Fi MAT 文件路径
file_list = sorted([f for f in os.listdir(DATA_FOLDER) if f.endswith('.mat')])

X_list, y_list = [], []

for idx, file_name in enumerate(file_list):
    mat = loadmat(os.path.join(DATA_FOLDER, file_name))
    if 'data_Ineed' not in mat:
        print(f"Warning: {file_name} 没有 'data_Ineed' 变量！")
        continue
    data_arr = mat['data_Ineed'].T  # 转置为 [num_samples, length]
    X_list.append(data_arr)
    y_list.append(np.full(data_arr.shape[0], idx, dtype=np.int64))
    print(f"第{idx}个文件 ({file_name}) 的数据 shape (转置后): {data_arr.shape}")

X = np.vstack(X_list)
y = np.concatenate(y_list)
num_classes = len(np.unique(y))
print(f"总数据 shape: {X.shape}, 标签 shape: {y.shape}, 类别数: {num_classes}")

# ====================== 数据处理函数 ======================
def compute_doppler_shift(v_kmh, fc_hz):
    if not v_kmh: return 0
    c = 3e8
    v_mps = v_kmh / 3.6
    return fc_hz * v_mps / c

def add_complex_awgn(signal, snr_db):
    if snr_db is None:
        return signal
    power = np.mean(np.abs(signal)**2)
    noise_power = power / (10**(snr_db/10))
    noise_std = np.sqrt(noise_power/2)
    noise = noise_std * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
    return signal + noise

def apply_doppler_shift(signal, fd_hz, fs_hz):
    if fd_hz is None or fd_hz == 0:
        return signal
    t = np.arange(len(signal)) / fs_hz
    return signal * np.exp(1j * 2 * np.pi * fd_hz * t)

def process_signal_led_rff(sig_complex, use_log=False, wavelet='db6', level=6):
    # --------------------
    # 1. FFT 幅度谱
    # --------------------
    freq_sig = np.fft.fft(sig_complex)
    amp = np.abs(freq_sig)
    # 对称频谱，只取前半段
    amp = amp[:len(amp)//2]

    # --------------------
    # 2. 可选 log
    # --------------------
    if use_log:
        amp = np.log(amp + 1e-8)

    # --------------------
    # 3. 小波分解 + 低频置零 + 重构
    # --------------------
    coeffs = pywt.wavedec(amp, wavelet, level=level)
    coeffs[0] = np.zeros_like(coeffs[0])
    rec = pywt.waverec(coeffs, wavelet)
    rec = rec[:len(amp)]

    # --------------------
    # 4. 归一化
    # --------------------
    mu, sigma = rec.mean(), rec.std()
    if sigma < 1e-8:
        feat = (rec - mu).astype(np.float32)
    else:
        feat = ((rec - mu) / (sigma + 1e-8)).astype(np.float32)
    return feat


def preprocess_iq_dataset_led_rff(data_real, snr_db=SNR_dB, velocity_kmh=VELOCITY_KMH,
                                  fc_hz=FC, fs_hz=FS, use_log=USE_LOG,
                                  wavelet=WAVELET, level=WAVELET_LEVEL,
                                  add_noise=ADD_NOISE, add_doppler=ADD_DOPPLER):
    num_samples, sig_len = data_real.shape
    processed_feats = []
    data_complex = data_real.astype(np.complex64)
    fd_hz = compute_doppler_shift(velocity_kmh, fc_hz) if add_doppler else None

    for i in range(num_samples):
        sig = data_complex[i]
        if add_noise: sig = add_complex_awgn(sig, snr_db)
        if add_doppler: sig = apply_doppler_shift(sig, fd_hz, fs_hz)
        # 调用新的 FFT + 小波处理
        feat = process_signal_led_rff(sig, use_log=use_log, wavelet=wavelet, level=level)
        processed_feats.append(feat)
    processed_feats = np.stack(processed_feats, axis=0)
    return torch.tensor(processed_feats, dtype=torch.float32)[:, None, :]  # [N, 1, length]


# ====================== InceptionTime 模型 ======================
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate=0.0):
        super().__init__()
        bottleneck_channels = max(1, out_channels // 4)

        self.bottleneck = nn.Conv1d(in_channels, bottleneck_channels, kernel_size=1, bias=False)

        self.conv1 = nn.Conv1d(bottleneck_channels, out_channels, kernel_size=10, padding=5)
        self.conv2 = nn.Conv1d(bottleneck_channels, out_channels, kernel_size=20, padding=10)
        self.conv3 = nn.Conv1d(bottleneck_channels, out_channels, kernel_size=40, padding=20)

        self.maxpool = nn.MaxPool1d(3, stride=1, padding=1)
        self.convpool = nn.Conv1d(bottleneck_channels, out_channels, kernel_size=1, bias=False)

        self.bn = nn.BatchNorm1d(4 * out_channels)
        self.relu = nn.ReLU()

        # ⭐ 新增 Dropout
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x_b = self.bottleneck(x)

        c1 = self.conv1(x_b)
        c2 = self.conv2(x_b)
        c3 = self.conv3(x_b)
        c4 = self.convpool(self.maxpool(x_b))

        min_len = min(c1.shape[-1], c2.shape[-1], c3.shape[-1], c4.shape[-1])
        c1 = c1[..., :min_len]
        c2 = c2[..., :min_len]
        c3 = c3[..., :min_len]
        c4 = c4[..., :min_len]

        out = torch.cat([c1, c2, c3, c4], dim=1)
        out = self.bn(out)
        out = self.relu(out)

        # ⭐ 加 dropout（inception block 内部正则化）
        out = self.dropout(out)
        return out


class InceptionTime(nn.Module):
    def __init__(self, num_classes, in_channels=1, channels=32, dropout_rate=0.0):
        super().__init__()

        self.b1 = InceptionBlock(in_channels, channels, dropout_rate)
        self.b2 = InceptionBlock(4 * channels, channels, dropout_rate)
        self.b3 = InceptionBlock(4 * channels, channels, dropout_rate)

        self.gap = nn.AdaptiveAvgPool1d(1)

        # ⭐ FC 之前也加 Dropout
        self.dropout = nn.Dropout(dropout_rate)

        self.fc = nn.Linear(4 * channels, num_classes)

    def forward(self, x):
        if x.shape[-1] % 2 == 1:
            x = x[..., :-1]

        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)

        x = self.gap(x).squeeze(-1)

        x = self.dropout(x)   # ⭐ dropout before classification

        return self.fc(x)

# ====================== 工具函数 ======================
def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix(cm, classes, fold, save_folder, dataset_type='Test'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{dataset_type} Confusion Matrix Fold{fold}')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(os.path.join(save_folder, f'{dataset_type.lower()}_cm_fold{fold}.png'))
    plt.close()

def plot_curves(train_losses, val_losses, train_acc, val_acc, fold, save_folder):
    plt.figure(); plt.plot(train_losses,label='Train Loss'); plt.plot(val_losses,label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title(f'Fold {fold} Loss'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder,f'loss_fold{fold}.png')); plt.close()
    plt.figure(); plt.plot(train_acc,label='Train Acc'); plt.plot(val_acc,label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.title(f'Fold {fold} Accuracy'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder,f'acc_fold{fold}.png')); plt.close()

# ====================== 划分训练集和测试集 ======================
X_proc = preprocess_iq_dataset_led_rff(X)
y_torch = torch.tensor(y, dtype=torch.long)

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_proc, y_torch, test_size=0.2, stratify=y_torch, random_state=42
)

# ====================== KFold 训练 ======================
def train_kfold(X_train, y_train, X_test, y_test, num_classes, device=DEVICE):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = f"{timestamp}_LED_WiFi_SNR{SNR_dB}dB_fd{int(compute_doppler_shift(VELOCITY_KMH, FC))}_classes_{num_classes}_CNN"
    save_folder = os.path.join(SAVE_ROOT, save_dir)
    os.makedirs(save_folder, exist_ok=True)
    results_file = os.path.join(save_folder, "results.txt")
    
    # 保存实验参数
    with open(results_file, 'w') as f:
        f.write(f"Timestamp: {timestamp}\nNum Classes: {num_classes}\nSNR_dB: {SNR_dB}\nVELOCITY_KMH: {VELOCITY_KMH}\nFC: {FC}\nFS: {FS}\nWavelet: {WAVELET}, Level: {WAVELET_LEVEL}\nBatch: {BATCH_SIZE}, Epochs: {EPOCHS}, LR: {LR}, WD: {WEIGHT_DECAY}\n\n")
    
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    full_dataset = TensorDataset(X_train, y_train)
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
    indices = np.arange(len(full_dataset))
    
    val_scores, test_scores = [], []
    
    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n=== Fold {fold+1}/{N_SPLITS} ===")
        tr_sub, va_sub = Subset(full_dataset, tr_idx), Subset(full_dataset, va_idx)
        tr_loader = DataLoader(tr_sub, batch_size=BATCH_SIZE, shuffle=True)
        va_loader = DataLoader(va_sub, batch_size=BATCH_SIZE, shuffle=False)
        
        model = InceptionTime(num_classes=num_classes, in_channels=1, channels=32, dropout_rate= DROPOUT_RATE).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        
        best_val, best_wts, patience_cnt = 0.0, None, 0
        train_losses, val_losses, train_acc_list, val_acc_list = [], [], [], []
        
        for epoch in range(EPOCHS):
            model.train(); running_loss, correct, total = 0.0, 0, 0
            for xb, yb in tr_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out,1)
                correct += (p==yb).sum().item()
                total += yb.size(0)
            train_loss = running_loss / len(tr_loader)
            train_acc = 100.0 * correct / total
            train_losses.append(train_loss)
            train_acc_list.append(train_acc)
            
            # Validation
            model.eval(); vloss, vcorrect, vtotal = 0.0,0,0
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb, yb = xb.to(device), yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out,1)
                    vcorrect += (p==yb).sum().item()
                    vtotal += yb.size(0)
            val_loss = vloss / len(va_loader)
            val_acc = 100.0 * vcorrect / vtotal
            val_losses.append(val_loss)
            val_acc_list.append(val_acc)
            
            print(f"Epoch {epoch+1}/{EPOCHS} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f}")
            with open(results_file, 'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f}\n")
            
            if val_acc > best_val + 0.1:
                best_val = val_acc
                best_wts = model.state_dict()
                patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= PATIENCE:
                    print("Early stopping.")
                    break
            scheduler.step()
        
        if best_wts is not None:
            model.load_state_dict(best_wts)
        
        # 测试集评估
        test_acc, test_cm = evaluate_model(model, test_loader, device, num_classes)
        print(f"Fold {fold+1} Test Accuracy: {test_acc:.2f}%")
        with open(results_file, 'a') as f:
            f.write(f"Fold {fold+1} TestAcc={test_acc:.2f}%\n")
        
        plot_confusion_matrix(test_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder)
        plot_curves(train_losses, val_losses, train_acc_list, val_acc_list, fold+1, save_folder)
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))
        
        val_scores.append(val_acc)
        test_scores.append(test_acc)
        
    # 总结
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")
    with open(results_file, 'a') as f:
        f.write(f"\n=== Overall Summary ===\nVal Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\nTest Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")
    
    
    print(f"\nAll results saved in {save_folder}")
    return save_folder

# ====================== 执行训练 ======================
# save_folder = train_kfold(X_train_val, y_train_val, X_test, y_test, num_classes=num_classes)

# ====================== 多 SNR 实验 ======================
SNR_list = list(range(20, -45, -5))   # 20, 15, 10, 5, 0, -5, ..., -40

all_snr_results = []

for SNR_dB in SNR_list:
    print("\n" + "="*70)
    print(f"[Experiment] Running SNR = {SNR_dB} dB")
    print("="*70)

    # ======= 重新处理特征 (每个 SNR 都要加噪声重新处理) =======
    X_proc = preprocess_iq_dataset_led_rff(
        X, 
        snr_db=SNR_dB,
        velocity_kmh=VELOCITY_KMH,
        fc_hz=FC,
        fs_hz=FS,
        use_log=USE_LOG,
        wavelet=WAVELET,
        level=WAVELET_LEVEL,
        add_noise=ADD_NOISE,
        add_doppler=ADD_DOPPLER
    )

    y_torch = torch.tensor(y, dtype=torch.long)

    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X_proc, y_torch, test_size=0.2, stratify=y_torch, random_state=42
    )

    # ======= 跑 5-Fold 训练 =======
    save_folder = train_kfold(
        X_train_val, 
        y_train_val, 
        X_test, 
        y_test, 
        num_classes=num_classes
    )

    all_snr_results.append((SNR_dB, save_folder))

print("\n\n================= 完成全部 SNR 实验 =================")
for snr, folder in all_snr_results:
    print(f"SNR={snr:>3} dB  →  结果目录: {folder}")

